# Load the data files from the Cornell Movie Dialog Corpus

 https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html


In [ ]:
import os

# Specify the path to the dataset files (adjust if needed)
data_dir = '../archive'

# Files we need from the corpus
lines_file = os.path.join(data_dir, 'movie_lines.txt')
conversations_file = os.path.join(data_dir, 'movie_conversations.txt')

# Check if the files exist
if os.path.exists(lines_file) and os.path.exists(conversations_file):
    print("Dataset files loaded successfully.")
else:
    print("Dataset files are missing. Please download and provide the correct paths.")

Dataset files loaded successfully.


# Read the lines from the movies

From README.txt

movie_lines.txt
	- contains the actual text of each utterance
	- fields:
		- lineID
		- characterID (who uttered this phrase)
		- movieID
		- character name
		- text of the utterance

Example:
L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".

In [ ]:
# Load the movie lines from the movie_lines file and create a dictionary
def load_lines(file_path):
    # Dictionary of Lines with Line ID and it's corresponding text
    lines = {}

    # with open(file_path, 'r', encoding='iso-8859-1') as f:
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            # Movie lines are in this format: LineID + Speaker + MovieID + Character + Text
            parts = line.strip().split(" +++$+++ ")
            # Perform an explicit check to ensure that the length of parts is exactly 5 before mapping the movie lines.
            # Avoid issues if any lines are malformed.
            if len(parts) == 5:
                line_id = parts[0]     #Line Id
                text = parts[4]        # Movie Line
                lines[line_id] = text  #Populate dictionary.
    return lines

# Load movie lines
lines = load_lines(lines_file)
print(f"Loaded {len(lines)} lines from the dataset.")

Loaded 304446 lines from the dataset.


# Read the conversations from the movies

    Check to see if these conversations' lines are read within movie_lines.
    If so, keep it.
  
- movie_conversations.txt
	- the structure of the conversations
	- fields
		- characterID of the first character involved in the conversation
		- characterID of the second character involved in the conversation
		- movieID of the movie in which the conversation occurred
		- list of the utterances that make the conversation, in chronological
			order: ['lineID1','lineID2',É,'lineIDN']
			has to be matched with movie_lines.txt to reconstruct the actual content

Example:
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']

In [ ]:
#Load the movie conversations
def load_conversations(file_path, lines):
    conversations = []

    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            # Movie conversations are in this format: Character1 + Character2 + MovieID + List of LineIDs
            parts = line.strip().split(" +++$+++ ")
            #retrieves line id's
            if len(parts) == 4:
                # Extract the list of line IDs and convert to text
                line_ids = eval(parts[3])  # eval to convert the string list to a list object
                conv = [lines[line_id] for line_id in line_ids if line_id in lines] # if line id matches entry in lines dictionary, keep it
                conversations.append(conv)
    return conversations

# Load conversations
conversations = load_conversations(conversations_file, lines)
print(f"Loaded {len(conversations)} conversations.")

print(conversations)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Exploratory Data analysis
# Text Cleaning

In [ ]:
def clean_sentence(sentence):
    # Remove punctuation
    sentence = re.sub(r"([.!?])", r" \1", sentence)
    # Remove non-letter characters
    sentence = re.sub(r"[^a-zA-Z.!?]+", r" ", sentence)
    return sentence

In [ ]:
import re
import nltk
import contractions # Removing contractions
import emoji # Convert Emoticons to Text if you want to perform sentiment analysis.
import unicodedata

#nltk.download('words')
words = set(nltk.corpus.words.words())

# Normalize string by converting unicode characters to ASCII and removing non-letters
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

# Function to convert emojis to words using emoji library mapping
def convert_emojis_to_words(text):
    converted_text = emoji.demojize(text)
    return converted_text

def remove_email(text):
    return re.sub(r'([a-z0-9+._-]+@[a-z0-9+_-]+)', "", text)

def remove_emojis(text):
    # Regular expression pattern to match emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Clitics
def clean_clitics(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    #text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return text

def remove_digitsInText(text):
    textWithoutDigits = list(filter(lambda x: x.isalpha(), text))
    return textWithoutDigits

def clean_text(text):
    # Step 1: Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    #Step 2: Remove emails
    text = remove_email(text)

    # Step 3: Remove Hashtags
    text = re.sub(r'#\w+', '', text)

    # Step 4: Remove Usernames (assuming they start with '@')
    text = re.sub(r'@\w+', '', text)

    # Step 5: Convert emojis to words
    text = convert_emojis_to_words(text)

    #Step 6: Remove any emoticons or emojis
    text = remove_emojis(text)

    # Step 7: Normalize Unicode characters to ASCII
    text = unicode_to_ascii(text.lower().strip())

    # Step 8: Remove punctuation, numbers, and extra spaces
    # Step 9: Remove any special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    #text = re.sub(r"[^\w\s]", '', text) #Remove special characters
    #text = remove_digitsInText(text) # Remove digits in text , Remove any words with digits like 5pm


    # Step 10: Convert to lowercase
    text = text.lower()

    # Step 11: Handle contractions / clitics
    text = contractions.fix(text)
    text = clean_clitics(text)

    # Step 12: Remove non-English words
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())

    return text

In [ ]:
from collections import Counter

class Vocabulary:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS", 3: "UNK"}
        self.num_words = 4  # Count SOS, EOS, UNK, PAD

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def sentence_to_indexes(self, sentence):
        return [self.word2index.get(word, 3) for word in sentence.split(' ')]  # 3 is UNK

# Example usage
vocab = Vocabulary()
vocab.add_sentence("hello how are you")
print(vocab.sentence_to_indexes("hello how are you"))


[4, 5, 6, 7]


In [ ]:
# Load and clean the data
def clean_and_prepare_data(lines_file, conversations_file):
    # lines = load_lines(lines_file)
    # conversations = load_conversations(conversations_file, lines)

    questions = []
    answers = []
    for conversation in conversations:
        for i in range(len(conversation) - 1):
            questions.append(conversation[i])
            answers.append(conversation[i + 1])

    clean_questions = [clean_text(q) for q in questions]
    clean_answers = [clean_text(a) for a in answers]

    clean_questions_to_indexes = []
    clean_answers_to_indexes = []

    # Filtering out short or long questions/answers
    # Keep only those sentences that have between 2 and 25 words.
    # Append the <EOS> (End of String) token to the end of each answer, indicating the end of the response for the model.
    filtered_questions, filtered_answers = [], []

    for q, a in zip(clean_questions, clean_answers):
        vocab.add_sentence(q)
        vocab.add_sentence(a)
        clean_questions_to_indexes.append(vocab.sentence_to_indexes(q))
        clean_answers_to_indexes.append(vocab.sentence_to_indexes(a))

        if 2 <= len(q.split()) <= 25 and 2 <= len(a.split()) <= 25:
            filtered_questions.append(q)
            filtered_answers.append(a + ' <EOS>')  # Append end token to answers

    return filtered_questions, filtered_answers

In [ ]:
# Load and clean the data
clean_questions, clean_answers = clean_and_prepare_data(lines_file, conversations_file)

# Print out the number of cleaned questions and answers
print(f"Number of cleaned questions: {len(clean_questions)}")
print(f"Number of cleaned answers: {len(clean_answers)}")

Number of cleaned questions: 152943
Number of cleaned answers: 152943


In [ ]:
# Validate the questions and answers
def validate_questions_answers(questions, answers, min_len=2, max_len=25):
    valid_questions = []
    valid_answers = []
    invalid_count = 0

    for question, answer in zip(questions, answers):
        # Check for non-empty strings and length constraints
        if isinstance(question, str) and isinstance(answer, str) and min_len <= len(question.split()) <= max_len and min_len <= len(answer.split()) <= max_len:
            valid_questions.append(question)
            valid_answers.append(answer)
        else:
            invalid_count += 1  # Count invalid pairs

    print(f"Total valid questions: {len(valid_questions)}")
    print(f"Total valid answers: {len(valid_answers)}")
    print(f"Total invalid pairs: {invalid_count}")

    return valid_questions, valid_answers

# Run validation
validated_questions, validated_answers = validate_questions_answers(clean_questions, clean_answers)

Total valid questions: 151703
Total valid answers: 151703
Total invalid pairs: 1240


# Train, Test Split before training using Cornell Movie corpus data

Train Set: Use this to train the model.

Validation Set: Use this to fine tune the models hyperparameter and evaluate the model during training. Monitor overfitting during training.

Test Set: Use this post training to test how well the model generalizes or to see how the model performs on unseen data.


In [ ]:
# from sklearn.model_selection import train_test_split

# # First split into train and test sets (80% train, 20% test)
# train_data, test_data = train_test_split(tokenized_conversations, test_size=0.2, random_state=42)

# # Further split the train data into train and validation sets (90% train, 10% validation)
# train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# # Output the sizes of each set
# print(f"Training set size: {len(train_data)}")
# print(f"Validation set size: {len(val_data)}")
# print(f"Test set size: {len(test_data)}")

Training set size: 111718
Validation set size: 12414
Test set size: 31033


In [ ]:
from sklearn.model_selection import train_test_split

# Split data into train, validation, and test sets
def split_data(clean_questions, clean_answers, test_size=0.2, val_size=0.1):
    # First split into train and remaining (which will be split further)
    questions_train, questions_rem, answers_train, answers_rem = train_test_split(
        clean_questions, clean_answers, test_size=(test_size + val_size), random_state=42)

    # Then split the remaining into validation and test
    val_size_adjusted = val_size / (test_size + val_size)
    questions_val, questions_test, answers_val, answers_test = train_test_split(
        questions_rem, answers_rem, test_size=val_size_adjusted, random_state=42)

    return (questions_train, answers_train), (questions_val, answers_val), (questions_test, answers_test)

(train_questions, train_answers), (val_questions, val_answers), (test_questions, test_answers) = split_data(validated_questions, validated_answers)

# Output the sizes of each set
print(f"Training set size: {len(train_questions)}")
print(f"Validation set size: {len(val_questions)}")
print(f"Test set size: {len(test_questions)}")


Training set size: 106192
Validation set size: 30340
Test set size: 15171


# Tokenization and Data Preparation

Use DialoGPT as the pre-trained model

Tokenization: The tokenizer from the Hugging Face Transformers library is used to convert the conversations into tokenized input for the model.

Managing Context: Need to ensure that previous conversation turns are included when generating a response.

Padding and Truncation: We need to ensure the inputs are padded or truncated to a fixed length for batch processing.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
from torch.utils.data import DataLoader, TensorDataset

# Load the pre-trained DialoGPT tokenizer
gpt_tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-small')

# Set the padding token to be the same as the end-of-sequence token
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

# Load the pre-trained DialoGPT model
gpt_model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')


# # Load DialoGPT tokenizer
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
# # Load DialoGPT model
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Load DialoGPT tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer.pad_token = tokenizer.eos_token

# Load DialoGPT model
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# # Function to tokenize conversations for multi-turn inputs
# def tokenize_conversation(conversations):
#     tokenized_data = []
#     for conversation in conversations:
#         # Tokenize and pad the conversation using DialoGPT tokenizer to a maximum length of 512
#         encoded_input = tokenizer.encode(' '.join(conversation), return_tensors='pt', max_length=512, padding='max_length', truncation=True)
#         tokenized_data.append(encoded_input)
#     return tokenized_data

# # Tokenize the conversations
# tokenized_conversations = tokenize_conversation(conversations)
# print(f"Tokenized {len(tokenized_conversations)} conversations.")

# Tokenize cleaned conversations
def tokenize_conversation(questions, answers, max_length=512):
    input_ids_list = []
    attention_masks_list = []
    labels_list = []

    for question, answer in zip(questions, answers):
        # Concatenate question and answer for tokenization
        encoded_input = tokenizer(question + " " + answer,
                                  return_tensors='pt',
                                  max_length=max_length,
                                  padding='max_length',  # Ensures padding up to max_length
                                  truncation=True)

        input_ids = encoded_input['input_ids']  # Don't squeeze here, handle batch dimension later
        attention_mask = encoded_input['attention_mask']

        # Skip empty inputs (if any)
        if input_ids.size(1) == 0:  # Check if input has a valid sequence length
            print(f"Empty input encountered for question-answer pair. Skipping.")
            continue  # Skip this pair if it's invalid

        # Set labels as input_ids with padding token ignored (-100)
        labels = input_ids.clone()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in labels

        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)
        labels_list.append(labels)

    # Stack input IDs, attention masks, and labels to create tensors
    return torch.cat(input_ids_list, dim=0), torch.cat(attention_masks_list, dim=0), torch.cat(labels_list, dim=0)

# Tokenization for the training data
train_input_ids, train_attention_masks, train_labels = tokenize_conversation(train_questions, train_answers)
val_input_ids, val_attention_masks, val_labels = tokenize_conversation(val_questions, val_answers)
test_input_ids, test_attention_masks, test_labels = tokenize_conversation(test_questions, test_answers)


In [ ]:
print(f"Tokenized Training {len(train_input_ids)} input id's conversations.")
print(f"Tokenized Training {len(train_attention_masks)} attention masks conversations.\n")

print(f"Tokenized Validation {len(val_input_ids)} input id's conversations.")
print(f"Tokenized Validation {len(val_attention_masks)} attention masks conversations.\n")

print(f"Tokenized Test {len(test_input_ids)} input id's conversations.")
print(f"Tokenized Test {len(test_attention_masks)} attention masks conversations.\n")

Tokenized Training 106192 input id's conversations.
Tokenized Training 106192 attention masks conversations.

Tokenized Validation 30340 input id's conversations.
Tokenized Validation 30340 attention masks conversations.

Tokenized Test 15171 input id's conversations.
Tokenized Test 15171 attention masks conversations.



# Create Data Loaders for the Train, Validation and Test Data

In [ ]:


from torch.utils.data import DataLoader, TensorDataset

# Create DataLoader with padding
def collate_fn(batch):
    return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True)

def create_data_loader(tokenized_data, batch_size=4):
    data_loader = DataLoader(tokenized_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    return data_loader

# Create dataset and DataLoader for batching
def create_dataloader(input_ids, attention_mask, labels, batch_size=16):
    dataset = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Running using CPU initially, so choosing batch size of 16
# Smaller batch size can lead to noisier gradient updates but sometimes result in better generalization.
# Larger batch sizes have more stable gradient updates but need more memory , learning rate needs tuning.
batch_size = 16

# # Create data loaders for the train, validation, and test sets
# train_loader = create_data_loader(train_data)
# val_loader = create_data_loader(val_data)
# test_loader = create_data_loader(test_data)

# print(f"Training loader size: {len(train_loader.dataset)}")
# print(f"Validation loader size: {len(val_loader.dataset)}")
# print(f"Test loader size: {len(test_loader.dataset)}")

# Create DataLoader for each dataset
train_dataloader = create_dataloader(train_input_ids, train_attention_masks, train_labels, batch_size=batch_size)
val_dataloader = create_dataloader(val_input_ids, val_attention_masks, val_labels, batch_size=batch_size)
test_dataloader = create_dataloader(test_input_ids, test_attention_masks, test_labels, batch_size=batch_size)

print(f"Training loader size: {len(train_dataloader.dataset)}")
print(f"Validation loader size: {len(val_dataloader.dataset)}")
print(f"Test loader size: {len(test_dataloader.dataset)}")

print(f"Number of steps in each epoch: {len(train_dataloader.dataset)/batch_size}")

Training loader size: 106192
Validation loader size: 30340
Test loader size: 15171
Number of steps in each epoch: 6637.0


# Validate Data After Tokenization:

Print out some tokenized examples to ensure that the tokenization process is working as expected, and there are no empty sequences or overly long sequences:

In [ ]:
for i, (input_ids, attention_mask, labels) in enumerate(train_dataloader):
    print(f"Batch {i} Tokenized Input Lengths: {[len(seq) for seq in input_ids]}")
    break  # Only print the first batch

Batch 0 Tokenized Input Lengths: [512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512]


# Training loop

Using manual training loop for better control over the training process instead of using HuggingFace's Trainer and TrainingArguments.


# Evaluation:

Evaluation metrics like BLEU score or ROUGE Score to measure the quality of the generated conversations.

BLEU Score: This metric is commonly used for evaluating machine translation and is calculated by comparing n-grams of the generated response against the reference responses.

ROUGE Score: This metric is often used for evaluating text summarization and compares the overlap of n-grams between the generated response and reference texts.


In [ ]:
import torch
from torch.optim import AdamW  # Import AdamW from PyTorch to avoid warning when imported using transformers
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time  # Import the time module

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Function to calculate BLEU score
def calculate_bleu(reference, candidate):
    # Tokenize the sentences
    reference_tokens = [ref.split() for ref in reference]  # Multiple reference sentences
    candidate_tokens = candidate.split()

    # Calculate BLEU score
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
    return bleu_score

# Function to calculate ROUGE score
def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores

In [ ]:
# Example usage
reference_responses = ["I am going to the store.", "I went to the store."]
generated_response = "I am going to the shop."

# Calculate BLEU score
bleu_score = calculate_bleu(reference_responses, generated_response)
print(f"BLEU Score: {bleu_score:.4f}")

# Calculate ROUGE score
rouge_scores = calculate_rouge(reference_responses[0], generated_response)  # Just using the first reference
print(f"ROUGE Scores: {rouge_scores}")

BLEU Score: 0.7598
ROUGE Scores: [{'rouge-1': {'r': 0.8333333333333334, 'p': 0.8333333333333334, 'f': 0.8333333283333335}, 'rouge-2': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002}, 'rouge-l': {'r': 0.8333333333333334, 'p': 0.8333333333333334, 'f': 0.8333333283333335}}]


In [ ]:
# Define validation function with accuracy, precision, recall, and F1-score
def validate_model(model, val_loader):
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            # Unpack the batch and move to CPU
            input_ids = batch.squeeze().to(device)

            # Perform a forward pass and calculate loss
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            total_val_loss += loss.item()

            # Get predictions
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            # Flatten input_ids and predictions for evaluation
            all_preds.extend(predictions.flatten().cpu().numpy())
            all_labels.extend(input_ids.flatten().cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

    # Calculate BLEU and ROUGE scores
    # For demonstration, you may want to keep some expected responses for evaluation
    # Here you can use a sample of generated responses
    # Note: In a real scenario, you would compare the generated response against actual reference responses
    # reference_responses = ["expected response 1", "expected response 2"]
    # generated_response = "Your model's generated response here."  # Replace with actual output from your model

    # bleu_score = calculate_bleu(reference_responses, generated_response)
    # print(f"BLEU Score: {bleu_score:.4f}")

    # rouge_scores = calculate_rouge(reference_responses[0], generated_response)  # Just using the first reference
    # print(f"ROUGE Scores: {rouge_scores}")


In [ ]:
def check_for_nan(tensor, name="tensor"):
    if torch.isnan(tensor).any():
        print(f"Found NaN values in {name}.")
        return True
    return False

In [ ]:

# Set device to CPU (since no GPU is available)
# device = torch.device('cpu')

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the CPU
model.to(device)

# Set up the optimizer (AdamW is a common choice for transformer models)
# Disable weight decay
optimizer = AdamW(model.parameters(), lr=1e-5,weight_decay=0.0) # Reduced the learning rate from 5e-5 due to nan loss


# Define training function
def train_model(model, train_dataloader, val_dataloader, optimizer, epochs=3):

    model.train()  # Set model to training mode

    for epoch in range(epochs):
        total_train_loss = 0

        print(f"\nEpoch {epoch + 1}/{epochs}:")

        start_time = time.time()  # Start time for the epoch

        # Training loop on  Train Data
        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            # Unpack the batch and move to the CPU device
            # input_ids = batch.squeeze().to(device)
            input_ids, attention_mask, labels = batch # Assuming batch contains (input_ids, attention_mask, labels)

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # Check for NaN values in input IDs
            # if torch.isnan(input_ids).any():
            if check_for_nan(input_ids, "input_ids") or check_for_nan(attention_mask, "attention_mask"):
                print("Input IDs contain NaN values. Stopping training.")
                print("NaN detected in inputs, stopping training.")
                return

            # Perform a forward pass and calculate loss
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss

            # Check for NaN loss
            if torch.isnan(loss):
                print(f"NaN loss encountered at Step {step} in Epoch {epoch+1}")
                print(f"Input IDs: {input_ids}")
                print(f"Attention Mask: {attention_mask}")
                print(f"Labels: {labels}")
                return  # Stop training to prevent further NaN propagation

            total_train_loss += loss.item()

            # Backward pass to calculate gradients
            loss.backward()

            # Check if gradients are NaN
            for param in model.parameters():
                if param.grad is not None and torch.isnan(param.grad).any():
                    print("Encountered NaN gradients. Stopping training.")
                    return

            # To prevent Gradient explosion due to large batch sizes, implement gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update parameters
            optimizer.step()

            # Print every 10th step
            if step % 10 == 0 and step > 0:
                print(f"Epoch {epoch+1},  Step {step}: Loss = {loss.item():.4f}")

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

         # Calculate the time taken for the epoch
        elapsed_time = time.time() - start_time
        print(f"Time taken for epoch {epoch + 1}: {elapsed_time:.2f} seconds")

        # Validation after each epoch using validation data
        validate_model(model, val_dataloader)

In [ ]:
# Train the model with 3 epochs
train_model(model, train_dataloader, val_dataloader, optimizer, epochs=3)


Epoch 1/3:
Epoch 1,  Step 10: Loss = 10.6100
Epoch 1,  Step 20: Loss = 8.9878


In [ ]:
# After training is complete, save the model and tokenizer
output_dir = r'./fine_tuned_dialoGPT'  # Specify a directory where you want to save the model

# Save the model and tokenizer after training is complete
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer saved to {output_dir}")

# Inference:

Build a chatbot interface where the model generates responses based on user inputs and conversation history.

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pre-trained DialoGPT model and tokenizer
# model_name = "microsoft/DialoGPT-small"
model_name = r'./fine_tuned_dialoGPT'  # or path to your trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate a response using the trained DialoGPT model
def generate_response(message, history):
    # Tokenize the input message and convert it to input_ids
    new_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt').to(device)

    # Concatenate the new input with the history of the conversation
    bot_input_ids = new_input_ids
    if history:
        history_ids = tokenizer.encode(history, return_tensors='pt').to(device)
        bot_input_ids = torch.cat([history_ids, new_input_ids], dim=-1)

    # Generate a response
    response_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode the generated response and add it to the history
    response = tokenizer.decode(response_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Update the history
    history = history + " " + message + " " + response

    return response, history  # Return both response and updated history

# Initialize Gradio ChatInterface
demo = gr.ChatInterface(
    fn=generate_response,
    examples=[{"text": "Hello", "files": []}],
    title="ChatBot Powered by DialoGPT",
    description="A chatbot based on the DialoGPT model, fine-tuned for multi-turn conversations.",
    multimodal=False
)

# Launch the Gradio demo
demo.launch()


In [ ]:
# prompt: generate a summary report of this notebook

def generate_summary_report(notebook_content):
    """Generates a summary report of a Jupyter Notebook content."""

    # Placeholder for a more sophisticated summarization method
    # (e.g., using a pre-trained summarization model)
    summary = f"""
    This Jupyter Notebook appears to focus on fine-tuning a DialoGPT model for multi-turn conversations.
    It includes sections for data loading, processing, model training, validation, and inference using Gradio.

    Key aspects:
    - Tokenization of input data using Transformers library.
    - Data loading using PyTorch DataLoader.
    - Model training using a custom training loop and AdamW optimizer.
    - Validation using metrics such as accuracy, precision, recall, and F1-score.
    - Evaluation of generated responses using BLEU and ROUGE scores.
    - Chatbot interface using Gradio for user interaction.
    """
    return summary


# Assuming notebook_content is the text content of your notebook
# notebook_content = ...

# Generate summary report
# summary_report = generate_summary_report(notebook_content)

# print("Summary Report:\n", summary_report)
